In [46]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
import matplotlib.pyplot as plt
import re
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords 
stop_words = set(stopwords.words('english'))
import string

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import hamming_loss
from sklearn.metrics import log_loss
from sklearn.metrics import zero_one_loss
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import balanced_accuracy_score

from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

pd.set_option('display.max_colwidth',100)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [48]:
df = pd.read_csv('/content/drive/My Drive/MCM_News_Aggregator/uci-news-aggregator.csv',usecols=['TITLE', 'CATEGORY','PUBLISHER'])
df.head(10)

,TITLE,PUBLISHER,CATEGORY
0,"Fed official says weak data caused by weather, should not slow taper",Los Angeles Times,b
1,Fed's Charles Plosser sees high bar for change in pace of tapering,Livemint,b
2,US open: Stocks fall after Fed official hints at accelerated tapering,IFA Magazine,b
3,"Fed risks falling 'behind the curve', Charles Plosser says",IFA Magazine,b
4,Fed's Plosser: Nasty Weather Has Curbed Job Growth,Moneynews,b
5,Plosser: Fed May Have to Accelerate Tapering Pace,NASDAQ,b
6,Fed's Plosser: Taper pace may be too slow,MarketWatch,b
7,Fed's Plosser expects US unemployment to fall to 6.2% by the end of 2014,FXstreet.com,b
8,US jobs growth last month hit by weather:Fed President Charles Plosser,Economic Times,b
9,ECB unlikely to end sterilisation of SMP purchases - traders,Interactive Investor,b


In [49]:
from keras.utils.np_utils import to_categorical

#I do aspire here to have balanced classes
num_of_categories = 45639
shuffled = df.reindex(np.random.permutation(df.index))
e = shuffled[shuffled['CATEGORY'] == 'e'][:num_of_categories]
b = shuffled[shuffled['CATEGORY'] == 'b'][:num_of_categories]
t = shuffled[shuffled['CATEGORY'] == 't'][:num_of_categories]
m = shuffled[shuffled['CATEGORY'] == 'm'][:num_of_categories]
concated = pd.concat([e,b,t,m], ignore_index=True)
#Shuffle the dataset
concated = concated.reindex(np.random.permutation(concated.index))
concated['LABEL'] = 0

concated.loc[concated['CATEGORY'] == 'e', 'LABEL'] = 0
concated.loc[concated['CATEGORY'] == 'b', 'LABEL'] = 1
concated.loc[concated['CATEGORY'] == 't', 'LABEL'] = 2
concated.loc[concated['CATEGORY'] == 'm', 'LABEL'] = 3
print(concated['LABEL'][:10])
labels = to_categorical(concated['LABEL'], num_classes=4)
print(labels[:10])
if 'CATEGORY' in concated.keys():
    concated.drop(['CATEGORY'], axis=1)

154056    3
14874     0
147249    3
76059     1
93136     2
78885     1
24505     0
284       0
19620     0
170937    3
Name: LABEL, dtype: int64
[[0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]]


In [50]:
concated.CATEGORY.value_counts()

t    45639
e    45639
m    45639
b    45639
Name: CATEGORY, dtype: int64

In [51]:
concated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182556 entries, 154056 to 47458
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   TITLE      182556 non-null  object
 1   PUBLISHER  182556 non-null  object
 2   CATEGORY   182556 non-null  object
 3   LABEL      182556 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 7.0+ MB


In [52]:
concated['title_publish'] = concated.TITLE + " " + concated.PUBLISHER
concated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182556 entries, 154056 to 47458
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   TITLE          182556 non-null  object
 1   PUBLISHER      182556 non-null  object
 2   CATEGORY       182556 non-null  object
 3   LABEL          182556 non-null  int64 
 4   title_publish  182556 non-null  object
dtypes: int64(1), object(4)
memory usage: 8.4+ MB


In [53]:
concated.CATEGORY.value_counts()

t    45639
e    45639
m    45639
b    45639
Name: CATEGORY, dtype: int64

In [54]:
# Drop null
concated= concated.dropna(subset=['title_publish'])

In [55]:
concated.head(5)

,TITLE,PUBLISHER,CATEGORY,LABEL,title_publish
154056,Pistorius did not have 'mental disorder' during Steenkamp shooting,Business Standard,m,3,Pistorius did not have 'mental disorder' during Steenkamp shooting Business Standard
14874,Victoria Beckham unleashes her inner gardener for Vogue 10:25AM,Glamour.com,e,0,Victoria Beckham unleashes her inner gardener for Vogue 10:25AM Glamour.com
147249,Have eczema? It may reduce your skin cancer risk,Big News Network.com,m,3,Have eczema? It may reduce your skin cancer risk Big News Network.com
76059,"AT&T posts $17.9 billion in revenue, adds 625000 postpaid customers in Q1",Daniel Rubino,b,1,"AT&T posts $17.9 billion in revenue, adds 625000 postpaid customers in Q1 Daniel Rubino"
93136,Microsoft Could Face China Antitrust Probe,TechWeekEurope UK,t,2,Microsoft Could Face China Antitrust Probe TechWeekEurope UK


In [56]:
symbols = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', 
           ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', 
           '#', '*', '+', '\\', '•',  '~', '@', '£', '·', '_', 
           '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', 
           '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', 
           '½', 'à', '…', '“', '★', '”', '–', '●', 'â', '►', 
           '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', 
           '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '▒', '：', 
           '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', 
           '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', 
           '‘', '∞', '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', 
           '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', 
           '¹', '≤', '‡', '√', ]

def clean_symbol(text):
    text = str(text)
    for symbol in symbols:
        text = text.replace(symbol, ' ')
    return text

# remove symbols and punctuations 
concated['title_publish'] = concated['title_publish'].apply(lambda x: clean_symbol(x))

In [57]:
# Dictionary of short form words and mispellings
short_forms_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", 
                    "'cause": "because", "could've": "could have", "couldn't": "could not", 
                    "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                    "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", 
                    "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", 
                    "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
                    "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", 
                    "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", 
                    "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
                    "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", 
                    "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
                    "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", 
                    "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", 
                    "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", 
                    "she'd": "she would", "she'd've": "she would have", "she'll": "she will", 
                    "she'll've": "she will have", "she's": "she is", "should've": "should have", 
                    "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                    "so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", 
                    "that's": "that is", "there'd": "there would", "there'd've": "there would have", 
                    "there's": "there is", "here's": "here is","they'd": "they would", 
                    "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", 
                    "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", 
                    "we'd": "we would", "we'd've": "we would have", "we'll": "we will", 
                    "we'll've": "we will have", "we're": "we are", "we've": "we have", 
                    "weren't": "were not", "what'll": "what will", "what'll've": "what will have", 
                    "what're": "what are",  "what's": "what is", "what've": "what have", 
                    "when's": "when is", "when've": "when have", "where'd": "where did", 
                    "where's": "where is", "where've": "where have", "who'll": "who will", 
                    "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", 
                    "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", 
                    "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", 
                    "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have",
                    "y'all're": "you all are","y'all've": "you all have","you'd": "you would", 
                    "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", 
                    "you're": "you are", "you've": "you have"}

def clean_shortforms(text):
    clean_text = text
    for shortform in short_forms_dict.keys():
        if re.search(shortform, text):
            clean_text = re.sub(shortform, short_forms_dict[shortform], text)
    return clean_text

# fix short forms
concated['title_publish'] = concated['title_publish'].apply(lambda x: clean_shortforms(x))

In [58]:
concated['title_publish']  = concated['title_publish'] .str.replace('[^\w\s]',' ')# unpunctuate

In [59]:
import re
import string

def clean_text(text):
    #text = re.sub("[0-9]+"," ",text)
    text = text.lower() # lowercase text
    text = re.sub(' +',' ', text)
    tokens=re.split('\W+',text)
    text1 = " ".join([word for word in tokens if word not in stop_words and word not in string.punctuation])
    return text1

In [60]:
concated['clean_title_publish'] = concated['title_publish'].apply(lambda x:clean_text(x))

In [61]:
concated.head(5)

,TITLE,PUBLISHER,CATEGORY,LABEL,title_publish,clean_title_publish
154056,Pistorius did not have 'mental disorder' during Steenkamp shooting,Business Standard,m,3,Pistorius did not have mental disorder during Steenkamp shooting Business Standard,pistorius mental disorder steenkamp shooting business standard
14874,Victoria Beckham unleashes her inner gardener for Vogue 10:25AM,Glamour.com,e,0,Victoria Beckham unleashes her inner gardener for Vogue 10 25AM Glamour com,victoria beckham unleashes inner gardener vogue 10 25am glamour com
147249,Have eczema? It may reduce your skin cancer risk,Big News Network.com,m,3,Have eczema It may reduce your skin cancer risk Big News Network com,eczema may reduce skin cancer risk big news network com
76059,"AT&T posts $17.9 billion in revenue, adds 625000 postpaid customers in Q1",Daniel Rubino,b,1,AT T posts 17 9 billion in revenue adds 625000 postpaid customers in Q1 Daniel Rubino,posts 17 9 billion revenue adds 625000 postpaid customers q1 daniel rubino
93136,Microsoft Could Face China Antitrust Probe,TechWeekEurope UK,t,2,Microsoft Could Face China Antitrust Probe TechWeekEurope UK,microsoft could face china antitrust probe techweekeurope uk


In [62]:
import re
import string

def clean_text1(text):
    text = string.capwords(text, sep = None) 
    tokens=re.split('\W+',text)
    tokens = list(set(tokens))
    text1 = " ".join([lemmatizer.lemmatize(word) for word in tokens if (len(word) > 2 and len(word) < 14)])
    return text1

In [63]:
concated['clean_title_publish'] = concated['clean_title_publish'].apply(lambda x:clean_text1(x))

In [64]:
concated.head(5)

,TITLE,PUBLISHER,CATEGORY,LABEL,title_publish,clean_title_publish
154056,Pistorius did not have 'mental disorder' during Steenkamp shooting,Business Standard,m,3,Pistorius did not have mental disorder during Steenkamp shooting Business Standard,Pistorius Disorder Standard Steenkamp Shooting Business Mental
14874,Victoria Beckham unleashes her inner gardener for Vogue 10:25AM,Glamour.com,e,0,Victoria Beckham unleashes her inner gardener for Vogue 10 25AM Glamour com,Victoria Unleashes Vogue Glamour Gardener Beckham Com Inner 25am
147249,Have eczema? It may reduce your skin cancer risk,Big News Network.com,m,3,Have eczema It may reduce your skin cancer risk Big News Network com,May Cancer Big Risk Skin News Reduce Network Com Eczema
76059,"AT&T posts $17.9 billion in revenue, adds 625000 postpaid customers in Q1",Daniel Rubino,b,1,AT T posts 17 9 billion in revenue adds 625000 postpaid customers in Q1 Daniel Rubino,Billion Daniel 625000 Revenue Adds Customers Postpaid Posts Rubino
93136,Microsoft Could Face China Antitrust Probe,TechWeekEurope UK,t,2,Microsoft Could Face China Antitrust Probe TechWeekEurope UK,Antitrust China Face Could Microsoft Probe


In [65]:
concated.tail(5)

,TITLE,PUBLISHER,CATEGORY,LABEL,title_publish,clean_title_publish
159219,Study: Sober smartphone app 'makes a big difference' in alcoholics' recovery,Columbia Missourian,m,3,Study Sober smartphone app makes a big difference in alcoholics recovery Columbia Missourian,Big Missourian Difference Recovery App Makes Alcoholics Smartphone Columbia Sober Study
64332,"TD Bank Published Unchanged Mortgage Interest Rates on April 18, 2014",Morning News USA,b,1,TD Bank Published Unchanged Mortgage Interest Rates on April 18 2014 Morning News USA,Morning News Usa Mortgage Rates Published April Unchanged 2014 Bank Interest
157697,HHS Awards $500K to Nebraska in Affordable Care Act Mental Health Funding,1011now,m,3,HHS Awards 500K to Nebraska in Affordable Care Act Mental Health Funding 1011now,Affordable 1011now Health 500k Nebraska Act Awards Mental Care Hhs Funding
107934,Apple and Samsung dominate smartphone market,LetsGoDigital,t,2,Apple and Samsung dominate smartphone market LetsGoDigital,Market Apple Letsgodigital Dominate Smartphone Samsung
47458,Salesforcecom Announces June 26 Press Event with Royal Philips,IT Business Net,b,1,Salesforcecom Announces June 26 Press Event with Royal Philips IT Business Net,Philips Event Royal June Announces Business Net Salesforcecom Press


In [66]:
concated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182556 entries, 154056 to 47458
Data columns (total 6 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   TITLE                182556 non-null  object
 1   PUBLISHER            182556 non-null  object
 2   CATEGORY             182556 non-null  object
 3   LABEL                182556 non-null  int64 
 4   title_publish        182556 non-null  object
 5   clean_title_publish  182556 non-null  object
dtypes: int64(1), object(5)
memory usage: 9.7+ MB


In [67]:
%%time
from gensim.models import KeyedVectors

wv = KeyedVectors.load_word2vec_format("/content/drive/My Drive/MCM_News_Aggregator/FastText/wiki-news-300d-1M-subword.vec")


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


CPU times: user 4min 23s, sys: 2.72 s, total: 4min 26s
Wall time: 4min 27s


In [68]:
#printing similarity index
print(wv.most_similar('gbp'))

[('usd', 0.7759820222854614), ('rmb', 0.7554484605789185), ('1Bn', 0.7260279059410095), ('stg', 0.7203834056854248), ('dlrs', 0.7126718759536743), ('45M', 0.7050207853317261), ('pcm', 0.7044318318367004), ('1mn', 0.7041310667991638), ('70M', 0.7029395699501038), ('34M', 0.6977018117904663)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [69]:
#wv['Wilx']

In [70]:
#word averaging
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.syn0norm[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, clean_title_publish) for clean_title_publish in text_list])

In [71]:
#tokenization
def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [72]:
%%time
train, test = train_test_split(concated, test_size=0.3,shuffle=True)
test_tokenized = test.apply(lambda r: w2v_tokenize_text(r['clean_title_publish']), axis=1).values
train_tokenized = train.apply(lambda r: w2v_tokenize_text(r['clean_title_publish']), axis=1).values

CPU times: user 46.1 s, sys: 220 ms, total: 46.3 s
Wall time: 46.4 s


In [73]:
%%time
X_train_word_average = word_averaging_list(wv,train_tokenized)
X_test_word_average = word_averaging_list(wv,test_tokenized)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


CPU times: user 13.3 s, sys: 214 ms, total: 13.5 s
Wall time: 13.5 s


Log Regression

In [74]:
"""
# multi_class
multi_class = ['multinomial']

# solver
solver = ['newton-cg', 'sag', 'saga', 'lbfgs']
 
# class_weight
class_weight = ['balanced', None]

# penalty
penalty = ['l2']

# Create the random grid
random_grid = {
               'multi_class': multi_class,
               'solver': solver,
               'class_weight': class_weight,
               'penalty': penalty}

"""

"\n# multi_class\nmulti_class = ['multinomial']\n\n# solver\nsolver = ['newton-cg', 'sag', 'saga', 'lbfgs']\n \n# class_weight\nclass_weight = ['balanced', None]\n\n# penalty\npenalty = ['l2']\n\n# Create the random grid\nrandom_grid = {\n               'multi_class': multi_class,\n               'solver': solver,\n               'class_weight': class_weight,\n               'penalty': penalty}\n\n"

In [75]:
"""

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
# First create the base model to tune
lrc = LogisticRegression(random_state=8)

# Definition of the random search
random_search = RandomizedSearchCV(estimator=lrc,
                                   param_distributions=random_grid,
                                   n_iter=50,
                                   scoring='accuracy',
                                   cv=3, 
                                   verbose=1, 
                                   random_state=8)

# Fit the random search model
random_search.fit(X_train_word_average, train['LABEL'])

"""

"\n\nfrom sklearn.model_selection import RandomizedSearchCV\nfrom sklearn.model_selection import GridSearchCV\n# First create the base model to tune\nlrc = LogisticRegression(random_state=8)\n\n# Definition of the random search\nrandom_search = RandomizedSearchCV(estimator=lrc,\n                                   param_distributions=random_grid,\n                                   n_iter=50,\n                                   scoring='accuracy',\n                                   cv=3, \n                                   verbose=1, \n                                   random_state=8)\n\n# Fit the random search model\nrandom_search.fit(X_train_word_average, train['LABEL'])\n\n"

In [76]:
"""
print("The best hyperparameters from Random Search are:")
print(random_search.best_params_)
print("")
print("The mean accuracy of a model with these hyperparameters is:")
print(random_search.best_score_)

"""


'\nprint("The best hyperparameters from Random Search are:")\nprint(random_search.best_params_)\nprint("")\nprint("The mean accuracy of a model with these hyperparameters is:")\nprint(random_search.best_score_)\n\n'

In [77]:
# First create the base model to tune
logreg = LogisticRegression(n_jobs=-1,solver='sag',penalty='l2',multi_class='multinomial',class_weight='balanced')

# Fit the random search model
logreg.fit(X_train_word_average, train['LABEL'])

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='multinomial', n_jobs=-1,
                   penalty='l2', random_state=None, solver='sag', tol=0.0001,
                   verbose=0, warm_start=False)

In [78]:
y_pred_test_log=logreg.predict(X_test_word_average)
y_pred_train_log=logreg.predict(X_train_word_average)

In [79]:
d=accuracy_score(y_pred_test_log,test.LABEL)*100
print ('Test Accuracy',d)

e=accuracy_score(y_pred_train_log,train.LABEL)*100
print ('Train Accuracy',e)

Test Accuracy 85.37440429455694
Train Accuracy 85.60282966452512


In [80]:
bac=balanced_accuracy_score(y_pred_test_log,test.LABEL)*100
c_k_s=cohen_kappa_score(y_pred_test_log,test.LABEL)*100
mc=matthews_corrcoef(y_pred_test_log,test.LABEL)*100


print(c_k_s)
print(mc)
print(bac)

80.49846566333989
80.50164251220615
85.36729700232209


SGD

In [81]:
%%time
sgd_search = SGDClassifier(n_jobs=-1, max_iter=20,alpha=0.00001)
sgd_search.fit(X_train_word_average,  train['LABEL'])

y_pred_test_sgd = sgd_search.predict(X_test_word_average)
y_pred_train_sgd = sgd_search.predict(X_train_word_average)


CPU times: user 9.32 s, sys: 45.5 ms, total: 9.36 s
Wall time: 4.97 s


In [82]:
f=accuracy_score(y_pred_test_sgd,test.LABEL)*100
print ('Test Accuracy',f)

g=accuracy_score(y_pred_train_sgd,train.LABEL)*100
print ('Train Accuracy',g)

Test Accuracy 85.27215293881352
Train Accuracy 85.48310104938611


In [83]:
print(classification_report(test.LABEL, y_pred_test_sgd))

              precision    recall  f1-score   support

           0       0.86      0.92      0.89     13772
           1       0.83      0.81      0.82     13778
           2       0.83      0.82      0.82     13632
           3       0.89      0.86      0.88     13585

    accuracy                           0.85     54767
   macro avg       0.85      0.85      0.85     54767
weighted avg       0.85      0.85      0.85     54767



In [84]:
# title-category function
def title_to_category1(title):
    title=clean_text(title)
    title=clean_text1(title)
    title=clean_symbol(title)
    title=clean_shortforms(title)
    categories = {1 : 'business', 
                  2 : 'science and technology', 
                  0 : 'entertainment', 
                  3 : 'health'}
    pridicter1 = logreg.predict(word_averaging_list(wv,w2v_tokenize_text(title)))
    return categories[pridicter1[0]]

In [85]:
title_to_category1('12 family getaways in Ireland this summer')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'business'

In [86]:
# title-category function
def title_to_category2(title):
    title=clean_text(title)
    title=clean_text1(title)
    title=clean_symbol(title)
    title=clean_shortforms(title)
    categories = {1 : 'business', 
                  2 : 'science and technology', 
                  0 : 'entertainment', 
                  3 : 'health'}
    pridicter2 = sgd_search.predict(word_averaging_list(wv,w2v_tokenize_text(title)))
    return categories[pridicter2[0]]

In [87]:
title_to_category2('Whole four-star hotel up for grabs in charity raffle')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'entertainment'